In [1]:
import pandas as pd
import numpy as np

Load actual values of demand, solar output and market price. Also load the predicted bidding price and quantity.

In [2]:
demand=pd.read_csv("../Processed Data/Demand_Val.csv", header=None)
solar_out=pd.read_csv("../Processed Data/Solar_Val.csv", header=None)
market_price=pd.read_csv("../Processed Data/Price_Val.csv", header=None)
bid_price=pd.read_csv("../Output/BidPrice.csv", header=None)
bid_quantity=pd.read_csv("../Output/BidQuantity.csv", header=None)

In [3]:
bill=pd.DataFrame(np.random.rand(50,24))

Calculate the daily bill using the constraints given in the PS.

Case 1: If bid price is greater than the market price, we buy the corresponding quantity from the market and charge or discharge the battery according to the solar output and the total demand of the block. If demand is greater than combined bid quantity, solar and battery, we buy electricity from discom.

Case 2: If bid price is less than the market price, we loose the bid and have to take electricity from battery and solar output. If that is not sufficient, we have to buy it from discom.

In [4]:
battery=np.float64(0.0)
for i in range(50):
    for j in range(24):
        block_bill=np.float64(0.0)
        if(bid_price.loc[i,j]>market_price.loc[i,j]):
            block_bill=block_bill+bid_price.loc[i,j]*bid_quantity.loc[i,j]
            if(bid_quantity.loc[i,j]>demand.loc[i,j] or bid_quantity.loc[i,j]+solar_out.loc[i,j]>demand.loc[i,j]):
                if(battery<np.float64(25.0)):
                    if(bid_quantity.loc[i,j]-demand.loc[i,j]+solar_out.loc[i,j]<=np.float64(5.0)):
                        battery=battery+bid_quantity.loc[i,j]-demand.loc[i,j]+solar_out.loc[i,j]
                    else:
                        battery=battery+np.float64(5.0)
                    if(battery>np.float64(25.0)):
                        battery=np.float64(25.0)
                else:
                    battery=np.float64(25.0)
            else:
                if(battery<=np.float64(5.0)):
                    if(demand.loc[i,j]>bid_quantity.loc[i,j]+solar_out.loc[i,j]+np.float64(0.8)*battery):
                        block_bill=block_bill+(np.float64(7.0)*(demand.loc[i,j]-bid_quantity.loc[i,j]-solar_out.loc[i,j]-np.float64(0.8)*battery))
                        battery=np.float64(0.0)
                    else:
                        battery=battery-(np.float64(1.25)*(demand.loc[i,j]-bid_quantity.loc[i,j]-solar_out.loc[i,j]))
                else:
                    if(demand.loc[i,j]>bid_quantity.loc[i,j]+solar_out.loc[i,j]+np.float64(4.0)):
                        block_bill=block_bill+(np.float64(7.0)*(demand.loc[i,j]-bid_quantity.loc[i,j]-solar_out.loc[i,j]-np.float64(4.0)))
                        battery=battery-np.float64(5.0)
                    else:
                        battery=battery-(np.float64(1.25)*(demand.loc[i,j]-bid_quantity.loc[i,j]-solar_out.loc[i,j]))
        else:
            if(solar_out.loc[i,j]>demand.loc[i,j]):
                if(battery<np.float64(25.0)):
                    if(solar_out.loc[i,j]-demand.loc[i,j]<=np.float64(5.0)):
                        battery=battery+solar_out.loc[i,j]-demand.loc[i,j]
                    else:
                        battery=battery+np.float64(5.0)
                    if(battery>np.float64(25.0)):
                        battery=np.float64(25.0)
                else:
                    battery=np.float64(25.0)
            else:
                if(battery<=np.float64(5.0)):
                    if(demand.loc[i,j]>solar_out.loc[i,j]+np.float64(0.8)*battery):
                        block_bill=block_bill+(np.float64(7.0)*(demand.loc[i,j]-solar_out.loc[i,j]-np.float64(0.8)*battery))
                        battery=np.float64(0.0)
                    else:
                        battery=battery-(np.float64(1.25)*(demand.loc[i,j]-solar_out.loc[i,j]))
                else:
                    if(demand.loc[i,j]>solar_out.loc[i,j]+np.float64(4.0)):
                        block_bill=block_bill+(np.float64(7.0)*(demand.loc[i,j]-solar_out.loc[i,j]-np.float64(4.0)))
                        battery=battery-np.float64(5.0)
                    else:
                        battery=battery-(np.float64(1.25)*(demand.loc[i,j]-solar_out.loc[i,j]))
        bill.loc[i,j]=block_bill

In [5]:
bill.to_csv("Daily_bill.csv", index=False, header=None)